In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import scipy as sp
import math
import sklearn
import plotly.graph_objects as go

### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

1. Carica il dataset.

In [16]:
df = pd.read_csv("/Users/nicola/Git/ProgrammingLab2-Nenzi/esercizi/data/data.csv")
df.head()

,date,passengers
0,1949-01,112.0
1,1949-02,118.0
2,1949-03,132.0
3,1949-04,129.0
4,1949-05,121.0


2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.

In [17]:
df["mese_numerico"] = np.arange(len(df))
df.head()

,date,passengers,mese_numerico
0,1949-01,112.0,0
1,1949-02,118.0,1
2,1949-03,132.0,2
3,1949-04,129.0,3
4,1949-05,121.0,4


3. Applica una regressione polinomiale (grado a tua scelta).

In [18]:
df_clean = df[["mese_numerico", "passengers"]].dropna()
X = df_clean[["mese_numerico"]]
y = df_clean["passengers"]
grado = 3
poly = sklearn.preprocessing.PolynomialFeatures(degree=grado)
X_poly = poly.fit_transform(X)

model = sklearn.linear_model.LinearRegression()
model.fit(X_poly, y)

LinearRegression()

4. Calcola l’RMSE tra i valori reali e quelli predetti.

In [19]:
preds = model.predict(X_poly)
rmse = sklearn.metrics.mean_squared_error(y, preds)
print(f"RMSE: {rmse:.2f}")

RMSE: 2027.54


5. Visualizza i dati reali e la curva stimata con Plotly.

In [20]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.loc[df_clean.index, "date"], y=y, mode="markers+lines", name="Dati reali"))
fig.add_trace(go.Scatter(x=df.loc[df_clean.index, "date"], y=preds, mode="lines", name="Regressione polinomiale"))
fig.update_layout(title="Regressione polinomiale sul numero di passeggeri",
                  xaxis_title="Data", yaxis_title="Numero di passeggeri")
fig.show()

### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [21]:
import dash
from dash import dcc, html, Input, Output

1. Genera 100 punti x tra -3 e 3.

In [22]:
np.random.seed(42)
X = np.linspace(-3, 3, 100)
y = X**3 - X + np.random.normal(scale=3, size=X.shape)

2. Calcola ad esempio y = x³ - x + rumore.

In [25]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Slider(
        id='grado-slider',
        min=1,
        max=10,
        step=1,
        value=3,
        marks={i: str(i) for i in range(1, 11)},
        tooltip={"placement": "bottom", "always_visible": True},
    ),
    dcc.Graph(id='grafico-polinomiale')
])

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.
4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [26]:
@app.callback(
    Output('grafico-polinomiale', 'figure'),
    Input('grado-slider', 'value')
)
def aggiorna_grafico(grado):
    poly = sklearn.preprocessing.PolynomialFeatures(degree=grado)
    X_poly = poly.fit_transform(X.reshape(-1, 1))
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_poly, y)
    y_pred = model.predict(X_poly)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X, y=y, mode='markers', name='Dati originali'))
    fig.add_trace(go.Scatter(x=X, y=y_pred, mode='lines', name=f'Polinomio grado {grado}'))
    fig.update_layout(title="Regressione Polinomiale", xaxis_title="X", yaxis_title="Y")
    return fig

if __name__ == '__main__':
    app.run(debug=True)